In [4]:
import sys
sys.path.insert(0, '/VIS_3TH/')

import numpy as np
from numba import njit

from setup import SHORT_PATH
import importlib.util
game_name = 'Exploding_Kitten'

def add_game_to_syspath(game_name):
    if len(sys.argv) >= 2:
        sys.argv = [sys.argv[0]]
    sys.argv.append(game_name)

def setup_game(game_name):
    spec = importlib.util.spec_from_file_location('env', f"{SHORT_PATH}Base/{game_name}/env.py")
    module = importlib.util.module_from_spec(spec)
    sys.modules[spec.name] = module 
    spec.loader.exec_module(module)
    return module

add_game_to_syspath(game_name)
env = setup_game(game_name)

getActionSize = env.getActionSize
getStateSize = env.getStateSize
getAgentSize = env.getAgentSize

getValidActions = env.getValidActions
getReward = env.getReward
numba_main_2 = env.numba_main_2

In [5]:
@njit()
def player(state, per):
    validActions = getValidActions(state)
    validActions = np.where(validActions == 1)[0]

    for i in (8, 7):
        if i in validActions and state[25]<10 and np.min(state[87:91])<5:
            return i, per

    if state[11]>0:
        if 6 in validActions:
            return 6, per
        
    if np.argmax(state[87:91])+11 in validActions:
        return np.argmax(state[87:91])+11, per
    
    for i in (5, 10, 9, 8, 7, 6, 1, 2, 4, 3, 0, 11):
        if i+15 in validActions:
            return i+15, per
    if np.sum(state[6:11])>0:
        for i in (6, 7, 8, 9, 10):
            if i+15 in validActions:
                return i+15, per
    for i in (11, 0, 3, 4, 2, 1):
        if i+27 in validActions:
            return i+27, per
    if np.sum(state[6:11])>0:
        for i in (6, 7, 8, 9, 10):
            if i+27 in validActions:
                return i+27, per
    for i in (11, 1, 2, 4, 3):
        if i+39 in validActions:
            return i+39, per

    for i in (3, 9):
        if i in validActions:
            return i, per
     
    if state[26]==2:
        for i in (1, 2, 0, 4):
            if i in validActions:
                return i, per
            
    action = validActions[-1]
    return action, per

In [6]:
for i in range(10):
    win, per = numba_main_2(player, 10000, np.array([]), 1)
    print(i, ":", win)

0 : 3517
1 : 3618
2 : 3525
3 : 3527
4 : 3664
5 : 3654
6 : 3572
7 : 3546
8 : 3597
9 : 3604
